In [1]:
import numpy as np
import gym
import copy
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision
import pandas as pd
import re

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [2]:
def normalizeQValues(arr, t_min, t_max):
    norm_arr = []
    diff = t_max - t_min
    diff_arr = max(arr) - min(arr)   
    for i in arr:
        temp = (((i - min(arr))*diff)/diff_arr) + t_min
        norm_arr.append(temp)
    return norm_arr

In [3]:
def normalize(arr):
    norm_arr = []
    max = 0
    indexMax = 0
    for a in range(len(arr)):
        if arr[a] >= max:
            max = arr[a]
            indexMax = a
    
    for a in range(len(arr)):
        if(a == indexMax):
            norm_arr.append(1)
        else:
            norm_arr.append(0)
    return norm_arr


In [4]:
def make2Dmatrix(arr):
    turn = arr[-1]
    arr = arr[:-1]
    for a in range(len(arr)):
        if(arr[a] != 0):
            arr[a] = arr[a]*turn

    arr = arr.reshape((5,5))
    
    return arr


In [5]:
#Hva må jeg gjøre med dataen?? lage x og y train og trening/test dataset.
f = open("gameStateData2.txt", "r")
data = f.read()
f.close()
gameStates = []
gameValuations = []
ingamestate = False
first = True
s = ""
m = ""
for i in range(len(data)):
    if i == len(data)-1:
        ingamestate = True
        first = False
        numbers = re.findall('-?\d+\.?\d*',str(m))
        listOfNumbersV = []
        for n in numbers:
            listOfNumbersV.append(float(n))
        m = ""
        if(listOfNumbersV != []):
            #listOfNumbersV = normalize(listOfNumbersV)
            listOfNumbersV = normalizeQValues(listOfNumbersV,0,1)
        gameValuations.append(listOfNumbersV)
    if data[i] == "T":
        gameStates[-1].append(float(1))
    elif data[i] == "F":
        gameStates[-1].append(float(-1))
    if data[i] == "[" and not ingamestate:
        ingamestate = True
        first = False
        numbers = re.findall('-?\d+\.?\d*',str(m))
        listOfNumbersV = []
        for n in numbers:
            listOfNumbersV.append(float(n))
        m = ""
        if(listOfNumbersV != []):
            #listOfNumbersV = normalize(listOfNumbersV)
            listOfNumbersV = normalizeQValues(listOfNumbersV,0,1)
        gameValuations.append(listOfNumbersV)
    elif data[i] == "[" and ingamestate:
        ingamestate = False
        numbers = re.findall('-?\d+\.?\d*',str(s))
        listOfNumbers = []
        for n in numbers:
            listOfNumbers.append(float(n))
        s = ""
        gameStates.append(listOfNumbers)
    if ingamestate:
        s += data[i]
    if not ingamestate and not first:
        m += data[i]

# gameStates is all the x input and whos turn it is
# gameValuations is all y output

gameValuations.remove([])
gameStates = np.asarray(gameStates)
newData = []


for a in range(len(gameStates)):
    newData.append(make2Dmatrix(gameStates[a]))
#print(gameValuations)
gameStates = newData

gameStates = torch.tensor(gameStates) #The array seems a bit weird
gameValuations = torch.tensor(np.asarray(gameValuations))

#f = open("reOrgDAta.txt", "w")
#for a in range(len(gameStates)):
#    f.write(str(a) + " : " + str(gameStates[a]) + " ::: " + str(gameValuations[a]) + "\n \n")
#f.close()

#print(y_test)

C:\Users\ugles\AppData\Local\Temp\ipykernel_60676\3170694999.py:66: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:233.)
  gameStates = torch.tensor(gameStates) #The array seems a bit weird


In [6]:
x_train = gameStates[int(np.floor(len(gameStates)/5)):].to(device= device, dtype=torch.float)
#x_train = gameStates[int(np.floor(len(gameStates)/5)):]
y_train = gameValuations[int(np.floor(len(gameValuations)/5)):].to(device= device, dtype=torch.float)
#y_train = gameValuations[int(np.floor(len(gameValuations)/5)):]
x_test = gameStates[:int(np.floor(len(gameStates)/5))].to(device= device, dtype=torch.float)
#x_test = gameStates[:int(np.floor(len(gameStates)/5))]
y_test = gameValuations[:int(np.floor(len(gameValuations)/5))].to(device= device, dtype=torch.float)
#y_test = gameValuations[:int(np.floor(len(gameValuations)/5))]

x_train = x_train.reshape(-1,1,5,5)
x_test = x_test.reshape(-1,1,5,5)

def find_max_list(list):
    list_len = [len(i) for i in list]
    print(min(list_len))

#print output#
find_max_list(y_train)

print(x_train)
print(len(x_train))
batches = 3251
x_train_batches = torch.split(x_train, batches)
y_train_batches = torch.split(y_train, batches)

26
tensor([[[[ 0.,  0., -1.,  1.,  0.],
          [ 0., -1., -1.,  1.,  0.],
          [ 0., -1.,  1.,  1., -1.],
          [-1.,  1.,  1.,  0.,  1.],
          [ 0., -1.,  0.,  1.,  0.]]],


        [[[ 0.,  0.,  1., -1.,  0.],
          [ 0.,  1.,  1., -1., -1.],
          [ 0.,  1., -1., -1.,  0.],
          [ 1., -1., -1.,  0., -1.],
          [ 0.,  1.,  0., -1.,  0.]]],


        [[[ 0.,  0., -1.,  1.,  0.],
          [ 0., -1., -1.,  1.,  1.],
          [ 0., -1.,  1.,  1.,  0.],
          [-1.,  1.,  1.,  0.,  1.],
          [ 0., -1., -1.,  1.,  0.]]],


        ...,


        [[[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0., -1.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  1.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]]],


        [[[ 0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  1.,  0.,  0.],
          [ 0.,  0.,  0., -1.,  0.],
          [ 0.,  0., -1.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.]]],


        [[[ 0.,  0.,  0.,  0.,  0.],


In [7]:
## NN, omg help, what am i doing!

class ConvolutionalNeuralNetworkModel(nn.Module):

    def __init__(self):
        super(ConvolutionalNeuralNetworkModel, self).__init__()

        # Model layers (includes initialized model variables):
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()

        )

        self.classifier = nn.Sequential(
            nn.Linear(64 * 5 * 5, 26),
            #nn.LogSoftmax(1)
            
        )
    # Predictor
    def forward(self, x):
        x = self.features(x)
        #print(x.shape)
        x = torch.flatten(x,1) #SJEKK OM BATCHENE GÅR OPP MED TOTAL INPUTEN
        #print(x.shape)
        x = self.classifier(x)
        return (x)

    # Cross Entropy loss
    def loss(self, x, y):
         return nn.functional.cross_entropy(self(x), y.argmax(1))

    #def loss(self, x, y):
    #    totalloss = 0
    #    for xi in range(len(self(x))): 
    #        print(self(x)[xi])
    #        print(y[xi])
    #        totalloss += (self(x)[xi]-y[xi])**2
    #        print("xi: " +  str(self(x)))
    #        print("yi: " + str(y[xi]))
    #    return torch.tensor(totalloss)

    # Accuracy
    def accuracy(self, x, y):
        return torch.mean(torch.eq(self(x).argmax(1), y.argmax(1)).float())


model = ConvolutionalNeuralNetworkModel()
model.to(device)

ConvolutionalNeuralNetworkModel(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
  )
  (classifier): Sequential(
    (0): Linear(in_features=1600, out_features=26, bias=True)
  )
)

In [8]:

# Optimize: adjust W and b to minimize loss using stochastic gradient descent
optimizer = torch.optim.Adam(model.parameters(), 0.001)
for epoch in range(2500):
    for batch in range(len(x_train_batches)):
        # Compute loss gradients
        loss = model.loss(x_train_batches[batch], y_train_batches[batch])
        loss.backward()
        optimizer.step()  # Perform optimization by adjusting W and b,
        optimizer.zero_grad()  # Clear gradients for next step

    if(epoch%250 == 0):
        print("accuracy = %s" % model.accuracy(x_test, y_test))

torch.save(model, "model")


accuracy = tensor(0.0808, device='cuda:0')
accuracy = tensor(0.2463, device='cuda:0')
accuracy = tensor(0.2463, device='cuda:0')
accuracy = tensor(0.2463, device='cuda:0')
accuracy = tensor(0.2429, device='cuda:0')
accuracy = tensor(0.2380, device='cuda:0')
accuracy = tensor(0.2380, device='cuda:0')
accuracy = tensor(0.2326, device='cuda:0')
accuracy = tensor(0.2302, device='cuda:0')
accuracy = tensor(0.2321, device='cuda:0')


In [9]:
newmodel = ConvolutionalNeuralNetworkModel()
newmodel = torch.load("model")
newmodel.eval()
print(x_test[5])
print(y_test[5])
with torch.no_grad():
    y = newmodel(x_test[5].unsqueeze(0))
    
print(torch.softmax(y,1))
print(device)



tensor([[[ 0.,  0.,  0.,  0.,  0.],
         [ 0.,  0., -1.,  0.,  0.],
         [ 0., -1., -1.,  1.,  0.],
         [ 0.,  1.,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.]]], device='cuda:0')
tensor([0.3379, 0.5068, 1.0000, 0.3379, 0.0913, 0.6712, 0.5023, 0.0000, 0.5890,
        0.5023, 0.5845, 0.0000, 0.0000, 0.0000, 0.6712, 0.5890, 0.0000, 0.0000,
        0.2603, 0.7489, 0.3425, 0.5890, 0.2557, 0.3425, 0.5023, 0.0959],
       device='cuda:0')
tensor([[3.5380e-11, 6.0569e-05, 1.3674e-06, 2.3591e-07, 3.4740e-12, 1.1028e-10,
         2.3278e-06, 4.1341e-22, 2.5030e-03, 3.8469e-06, 1.0586e-05, 2.4711e-15,
         8.8550e-17, 9.2633e-22, 4.3732e-07, 9.9666e-01, 5.8090e-20, 6.3329e-21,
         7.4864e-04, 5.6865e-06, 3.1497e-14, 2.2587e-13, 6.6197e-09, 2.2139e-08,
         4.1945e-12, 1.4174e-17]], device='cuda:0')
cuda
